<h1>Spotted samples</h1>
<h3>Analyze peak intensity for spotted MSI data</h3>

In [1]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ast
import requests, json
import sys
sys.path.append( '/Users/bpb/repos/openmsi-programmatic-access/' )
import openmsi
import openmsi_spotset_tools
# plt.get_backend()
# plt.switch_backend('Qt4Agg')
client = requests.Session()

<h1>Authenticate to NERSC</h1>
<h3>Put your NERSC username in the quotes below and enter password in the box.  Requires up to date IPython to work</h3>

In [2]:
client = openmsi.authenticateUser(client,'bpb') # put your username here in the quotes

##### Test that we are authenticated #####
##### by printing a list of files #####
##### you have access to #####
myFiles = openmsi.getFilelist(client)
for k in myFiles:
    print k
##### If you don't see private files. #####
##### It didn't authenticate.  #####

········
/project/projectdirs/openmsi/omsi_data_private/bpb/20110426_Liver_00hrs.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/Challenge_2W_v7.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/20140512_KBL_NP_EColi_cosmid_ACNex_super_pel_super_70SS_15nL.h5
/project/projectdirs/openmsi/omsi_data_private/raad0102/20150427MdR_Chemical_Screen_3_try2.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/20140828_KBL_4and2_topchip_2D2O_nodebri2_TI.h5
/project/projectdirs/openmsi/omsi_data_private/joshvh/20151104_GH1_G2.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/20151015_BCD_Liver_Test_Mouse_100ss.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/Flax_Pod_7_day_old_CS.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/20140801_myxovbsurf.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/20121210_RCH2MR1_1day_K_lines.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/20121012_lipid_extracts.h5
/project/projectdirs/openmsi/omsi_data_private/bpb/20140425_KBL_

<h1>Specify the full path to the file you want</h1>
<h3>You also have to specify the data and/or ana index</h3>
<h3>For multi-experiment files, you have to specify the experiment index</h3>

In [3]:
# filename = '/project/projectdirs/openmsi/omsi_data_private/bpb/20120913_nimzyme.h5'
filename = '/project/projectdirs/openmsi/omsi_data_private/joshvh/20151104_GH1_G2.h5'
# filename = '/project/projectdirs/openmsi/omsi_data_private/raad0102/20150331MdR_MALDI-NIMS_1.h5'
dataIndex = '0'
expIndex = '0' ### for single experiment files, the experiment index is zero ###
payload = {'file':filename,'format':'JSON','mtype':'file','expIndex':expIndex,'dataIndex':dataIndex}
url = 'https://openmsi.nersc.gov/openmsi/qmetadata'
r = client.get(url,params=payload)
metadata = json.loads(r.content)
originalSize = ast.literal_eval(metadata[u'children'][0][u'shape'])
print originalSize
### prints the [x, y, m/z] of the data cube

(13, 122, 138919)


<h1>Specify the ions for analysis</h1>

In [22]:
# myIons = [1241.25, 917.5, 817.6]
myIons = [1102.15, 940, 1102]
# myIons = [2093.1, 2465.2, 2867.0, 3494.7, 3657.9, 5735.0]
myRange = 0.05 # integrate at +/- this amount

<h1>Get the images for each ion of interest</h1>
<h3>Change the "np.sum" to "np.max" for peak height instead of peak area</h3>

In [31]:
mz = openmsi.getMZ(client,filename,expIndex,dataIndex)
imStack = np.zeros((originalSize[0],originalSize[1],len(myIons)))
# for each ion in myions, get an image for that ion
for i,ion in enumerate(myIons):
    idx = np.where(abs(mz-ion)<myRange) #get the m/z indices within myRange
    payload = {'file':filename,
               'expIndex':expIndex,'dataIndex':dataIndex,'format':'JSON','mz':'%d:%d'%(min(idx[0]),max(idx[0]))}
    url = 'https://openmsi.nersc.gov/openmsi/qcube'
    r = client.get(url,params=payload)
    data = np.asarray(json.loads(r.content))
    ### to do: add background subtraction for "peakless" data ###
#     imStack[:,:,i] = np.sum(data,2) #### Pick this one for peak area
    imStack[:,:,i] = np.max(data,2) #### Pick this one for peak height

##### SELECT ONE OF THE METHODS BELOW FOR MAKING BASEIMAGE #####
##### baseImage is used to make the visualization for placing the markers #####
##### 1.  Here baseImage is the sum intensity of all your ion images #####
# baseImage = np.sum(imStack,2)  #ALL IONS

##### 2.  Here baseImage is a specific ion images #####
# baseImage = imStack[:,:,0] #ONE ION

##### baseImage is used to make the visualization for placing the markers #####
##### 3.  Here baseImage is the sum intensity of specific ion images #####
myIdx = [0,1,2]
baseImage = np.sum(imStack[:,:,myIdx],2) #SPECIFIC IONS

plt.imshow(baseImage,cmap='CMRmap')
plt.show()

Set the circles where they need to be.  Then close the figure window. The last coordinates of the circles will be stored and you use them to study the interior points of your trapezoid.

In [32]:
Nrows = 6
Ncols = 1
dragRadius = 4
pointMarkerSize = 12
xCenter,yCenter = openmsi_spotset_tools.roughPosition(baseImage,Nrows,Ncols,dragRadius,pointMarkerSize)

In [25]:
markerRadius = 2
xCenter,yCenter = openmsi_spotset_tools.fineTunePosition(baseImage,xCenter,yCenter,markerRadius)

In [26]:
# Save a dictionary into a pickle file.

import pickle

spotset_data = {'xCenter':xCenter,'yCenter':yCenter,
            'Nrows':Nrows,'Ncols':Ncols,
            'dragRadius':dragRadius,'pointMarkerSize':pointMarkerSize,'markerRadius':markerRadius,
            'baseImage':baseImage,'imStack':imStack,'myIons':myIons,'myRange':myRange,'mz':mz,'originalSize':originalSize,
            'filename':filename,'expIndex':expIndex,'dataIndex':dataIndex}

pickle.dump( spotset_data, open( "myWorkspace.pkl", "wb" ) )

In [27]:
# Load the dictionary back from the pickle file.
import pickle

spotset_data = pickle.load( open( "myWorkspace.pkl", "rb" ) )
for key in spotset_data:
    globals()[key]=spotset_data[key]

In [28]:
xEdges, yEdges = np.meshgrid(range(imStack.shape[1]), range(imStack.shape[0]), sparse=False, indexing='xy')
mask = np.zeros(baseImage.shape)
integrationRadius = 6 #distance to integrate
myPixels = []
for x,y in zip(xCenter,yCenter):
    idx = np.argwhere(((x - xEdges)**2 + (y - yEdges)**2)**0.5 < integrationRadius)
    myPixels.append(idx)
    for i in idx:
        mask[i[0],i[1]] = 1
plt.imshow(mask)
plt.show()
    

In [29]:
minIntensity = 100

In [30]:
fid = open('export.tab','wb')
fid.write('index\tfile\trow\tcolumn\trow-centroid\tcol-centroid\t')
for i in myIons:
    fid.write('%5.4f Sum\t' % i)
    fid.write('%5.4f Max\t' % i)
    fid.write('%5.4f Mean\t' % i)
    fid.write('%5.4f Median\t' % i)
    fid.write('%5.4f NumPixels\t' % i)
fid.write('\n')

for i,myPixel in enumerate(myPixels): #how many spots
    fid.write('%d\t%s\t%s\t%s\t%d\t%d\t' % ( i, filename, 'coming_soon', 'coming_soon', np.mean(myPixel[:,0]), np.mean(myPixel[:,1]) ) )
    for i,ion in enumerate(myIons): #how many ions
        values = []
        for j, coord in enumerate(myPixel): #how many pixels per spot
            if imStack[coord[0],coord[1],i] > minIntensity:
                print imStack[coord[0],coord[1],i]
                print coord[0],coord[1],i
                #accumulate a list of peak height or 
                #peak area values for each pixel 
                #assigned to each spot
                values.append(imStack[coord[0],coord[1],i]) 
        if len(values) > 0:        
            fid.write('%d\t%d\t%d\t%d\t%d\t' % (np.sum(values),np.max(values),np.mean(values),np.median(values),len(values)))
        else:
            fid.write('%d\t%d\t%d\t%d\t%d\t' % (0,0,0,0,len(values)))
        
    fid.write('\n')
fid.close()

132.0
1 109 0
140.0
1 110 0
137.0
2 107 0
136.0
2 108 0
184.0
2 109 0
195.0
2 110 0
224.0
2 111 0
155.0
2 112 0
131.0
2 113 0
131.0
3 106 0
147.0
3 107 0
136.0
3 108 0
220.0
3 109 0
189.0
3 110 0
229.0
3 111 0
220.0
3 112 0
210.0
3 113 0
258.0
3 114 0
132.0
4 105 0
160.0
4 106 0
278.0
4 107 0
277.0
4 108 0
244.0
4 109 0
297.0
4 110 0
174.0
4 111 0
259.0
4 112 0
190.0
4 113 0
268.0
4 114 0
287.0
4 115 0
136.0
5 105 0
131.0
5 106 0
219.0
5 107 0
239.0
5 108 0
204.0
5 109 0
224.0
5 110 0
263.0
5 111 0
494.0
5 112 0
293.0
5 113 0
122.0
5 114 0
224.0
5 115 0
184.0
6 104 0
146.0
6 105 0
141.0
6 106 0
161.0
6 107 0
214.0
6 108 0
219.0
6 109 0
391.0
6 110 0
651.0
6 111 0
611.0
6 112 0
131.0
6 113 0
121.0
6 114 0
137.0
6 115 0
239.0
7 104 0
121.0
7 105 0
131.0
7 106 0
214.0
7 107 0
189.0
7 108 0
180.0
7 109 0
318.0
7 110 0
224.0
7 111 0
303.0
7 112 0
194.0
7 113 0
140.0
7 114 0
132.0
7 115 0
450.0
8 104 0
160.0
8 105 0
155.0
8 106 0
121.0
8 107 0
210.0
8 108 0
175.0
8 109 0
366.0
8 110 0
214.0


In [19]:
myPixels

[array([[  0, 107],
        [  0, 108],
        [  1, 105],
        [  1, 106],
        [  1, 107],
        [  1, 108],
        [  1, 109],
        [  1, 110],
        [  1, 111],
        [  2, 104],
        [  2, 105],
        [  2, 106],
        [  2, 107],
        [  2, 108],
        [  2, 109],
        [  2, 110],
        [  2, 111],
        [  2, 112],
        [  3, 103],
        [  3, 104],
        [  3, 105],
        [  3, 106],
        [  3, 107],
        [  3, 108],
        [  3, 109],
        [  3, 110],
        [  3, 111],
        [  3, 112],
        [  3, 113],
        [  4, 103],
        [  4, 104],
        [  4, 105],
        [  4, 106],
        [  4, 107],
        [  4, 108],
        [  4, 109],
        [  4, 110],
        [  4, 111],
        [  4, 112],
        [  4, 113],
        [  5, 102],
        [  5, 103],
        [  5, 104],
        [  5, 105],
        [  5, 106],
        [  5, 107],
        [  5, 108],
        [  5, 109],
        [  5, 110],
        [  5, 111],
